# L4: Optimize DSPy Agent with DSPy Optimizer

In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access your API keys
google_api_key = os.getenv('GOOGLE_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
# Use the keys
print(f"Google API Key loaded: {bool(google_api_key)}")
print(f"OpenAI API Key loaded: {bool(openai_api_key)}")

os.environ["OPENAI_API_KEY"]  = openai_api_key

Google API Key loaded: True
OpenAI API Key loaded: True


In [2]:
import mlflow

In [14]:
from pathlib import Path
notebook_path = Path.cwd()
root_path = Path.cwd().parent
data_path = root_path / "data"
mlflow_tracking_uri = notebook_path / "mlruns"
print(notebook_path)

e:\GIT_ROOT\Learning\DLAI-shortcourse_notebooks\courses\dspy\notebooks


In [3]:
def get_mlflow_tracking_uri():
    return "http://localhost:8080"
    #return os.environ.get('DLAI_LOCAL_URL').format(port=8080)


mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [ ]:
# !mlflow server --host 127.0.0.1 --port 8080

In [4]:
mlflow.set_experiment("dspy_course_4")

2025/07/06 18:47:22 INFO mlflow.tracking.fluent: Experiment with name 'dspy_course_4' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/840480777417013393', creation_time=1751807842407, experiment_id='840480777417013393', last_update_time=1751807842407, lifecycle_stage='active', name='dspy_course_4', tags={}>

In [5]:
mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [6]:
import dspy

dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Build a RAG Agent

In [7]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
import json

# Load trainset
trainset = []
with open(data_path/"trainset.jsonl", "r") as f:
    for line in f:
        trainset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

# Load valset
valset = []
with open(data_path/"valset.jsonl", "r") as f:
    for line in f:
        valset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

In [16]:
# Overview of the dataset.
print(trainset[0])

Example({'question': 'Are Smyrnium and Nymania both types of plant?', 'answer': 'yes'}) (input_keys={'question'})


In [19]:
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",
    num_threads=16
)

In [17]:
dspy.cache.load_memory_cache(data_path/"memory_cache.pkl")

In [20]:
optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/07/06 18:51:57 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4fb3da48f6a84886bb2d96948876c41e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/07/06 18:51:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/07/06 18:51:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/06 18:51:57 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/07/06 18:51:57 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 15%|█▌        | 15/100 [02:03<11:37,  8.21s/it]

Bootstrapped 4 full traces after 15 examples for up to 1 rounds, amounting to 15 attempts.


Bootstrapping set 4/6


...

2025/07/06 19:06:31 INFO dspy.evaluate.evaluate: Average Metric: 47 / 100 (47.0%)



🏃 View run eval_full_4 at: http://localhost:8080/#/experiments/840480777417013393/runs/184e5b87215b48489d82e6e2b0f03df3
🧪 View experiment at: http://localhost:8080/#/experiments/840480777417013393


2025/07/06 19:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [29.0, 51.0, 48.0, 49.0, 47.0]
2025/07/06 19:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 51.0
2025/07/06 19:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/06 19:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/06 19:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 51.0!


🏃 View run rogue-calf-148 at: http://localhost:8080/#/experiments/840480777417013393/runs/4fb3da48f6a84886bb2d96948876c41e
🧪 View experiment at: http://localhost:8080/#/experiments/840480777417013393


[Trace(trace_id=67e7671d74134d11b8e68a0a13aaa450), Trace(trace_id=aa49eaf53be64103ad400a26f5d63e89), Trace(trace_id=cccaf0dc5ca34725ab522ba5fe25db38), Trace(trace_id=4c85fb5702454d99ad8f3ff312359608), Trace(trace_id=75f90e27e0e441188a8c3716ec4d275a), Trace(trace_id=1ed4c174f4444c1797f190b79dba92a2), Trace(trace_id=b1ebf4fc4cee43a2a70b76e46a2ffeb8), Trace(trace_id=15fd71fc2680475c9557a46b446b8dab), Trace(trace_id=efd6800d7f834c118275ca20dd0e81fb), Trace(trace_id=f5fe4ca8a6104194beb066703d0e1717)]

In [ ]:
optimized_react.react.signature

In [21]:
optimized_react.react.demos

[Example({'augmented': True, 'question': 'Konnichiwa was launched in a broadcast that was streamed by the platform that was founded in what city in 2010?', 'trajectory': '[[ ## thought_0 ## ]]\nI need to find out which platform was founded in 2010 and in what city. This information will help me answer the question about the broadcast of Konnichiwa. I will search for platforms that were established in 2010.\n\n[[ ## tool_name_0 ## ]]\nsearch_wikipedia\n\n[[ ## tool_args_0 ## ]]\n{"query": "platform founded in 2010"}\n\n[[ ## observation_0 ## ]]\n[1] «Genius Crowds | Genius Crowds was a crowdsourcing platform founded in 2010 on the principles of collaboration and sharing. It served as a platform for everyday people – students, women, men, moms, dads, DIY\'ers, cooks, and tinkerers - who have a product idea that they don\'t know how to bring to market.»\n[2] «Seedups | SeedUps is an equity crowdfunding platform for seed investment to launch and focuses on startup companies. It was founded

In [22]:
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,
    devset=valset,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [23]:
original_score = evaluator(react)
print(f"Original score: {original_score}")

Average Metric: 29.00 / 100 (29.0%): 100%|██████████| 100/100 [00:12<00:00,  7.95it/s]

2025/07/06 19:06:47 INFO dspy.evaluate.evaluate: Average Metric: 29 / 100 (29.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...",Bud Ekins served as Steve McQueen's stunt double in the classic fi...,The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,"{'thought_0': 'I need to determine which individual, Robert Kardas...",Robert Kardashian's family is well-known for their reality TV show...,Robert Kardashian's family had their own reality TV show.,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in the film ""Shadows ...","I searched for information about the cast of the film ""Shadows in ...","There is no Russian ballerina in ""Shadows in Paradise.""",
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,{'thought_0': 'I need to gather information about the man who appo...,"The man who appointed Amashsai was Nehemiah, a biblical figure kno...","The meaning of the name of the man who appointed Amashsai, Nehemia...",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'To answer the question about what is needed in addi...,To gain access to 173 countries and territories with an Austrian p...,"In addition to the Austrian passport, travelers may need to obtain...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the name of the American actress...,The American actress and singer-songwriter known for her role as P...,2007,
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The question asks for animated creatures that were the title chara...,The animated creatures that were the title characters of the film ...,
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to verify the nationalities and contribution...,Both Dorothy Arzner and Richard Wallace were confirmed to be Ameri...,"No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run eval at: http://localhost:8080/#/experiments/840480777417013393/runs/c0848e006ff946a9b8f5425fc95d2f0e
🧪 View experiment at: http://localhost:8080/#/experiments/840480777417013393
Original score: 29.0


[Trace(trace_id=a2cdc48144be45d68d6079a4e0cd70e6), Trace(trace_id=9b71bf01392748e9b03c9f22c8cf176a), Trace(trace_id=1ab593821bca441198cf4c93d27be517), Trace(trace_id=afc695e08fbc4286a0abda40194ec4dd), Trace(trace_id=7616415043fc490cb4b8bbba410dffee), Trace(trace_id=ac42bcce593f4f29a3cd418552c6259b), Trace(trace_id=6cbba1a5dc1b4799ba7d9f3c1f7183a7), Trace(trace_id=77bc04cc84af4b3693fe3046472e2746), Trace(trace_id=64017b10c8794afc971d6bdf1ecb5dfe), Trace(trace_id=aead255c960b4f11a1b099a75bef42d5)]

In [24]:
optimized_score = evaluator(optimized_react)
print(f"Optimized score: {optimized_score}")

Average Metric: 51.00 / 100 (51.0%): 100%|██████████| 100/100 [00:15<00:00,  6.59it/s]

2025/07/06 19:07:03 INFO dspy.evaluate.evaluate: Average Metric: 51 / 100 (51.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...","Steve McQueen, referred to as ""the king of cool,"" starred in the f...",The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,"{'thought_0': 'I need to determine which of the two individuals, R...","Robert Kardashian is part of the Kardashian family, who are famous...",Robert Kardashian,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in the film ""Shadows ...","In the 2010 film ""Shadows in Paradise,"" Sofya Skya is a prominent ...",Sofya Skya,✔️ [True]
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...","Amashsai was appointed by Nehemiah, and the name ""Nehemiah"" means ...",Yahweh comforts,
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'To answer the question about what is needed in addi...,The information gathered indicates that Austrian citizens have vis...,Austrian passport (and possibly a visa for specific countries),
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the name of the American actress...,The American actress and singer-songwriter known for her role as P...,2007,
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The animated creatures that are the title characters of the film b...,"Fairies (Puck, Titania, Oberon)",
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to determine the nationalities of both Dorot...,"Dorothy Arzner was an American film director, and Richard Wallace ...","No, they were not both French film directors.",


🏃 View run eval at: http://localhost:8080/#/experiments/840480777417013393/runs/9abf7ebe6ed748f4b1cbb6772eafa5f9
🧪 View experiment at: http://localhost:8080/#/experiments/840480777417013393
Optimized score: 51.0


[Trace(trace_id=369d5e66fd7544b79850a0d0eaebbc55), Trace(trace_id=12f25055d89d48b2ac72ea1f825cad3c), Trace(trace_id=7096e461474e46f99551ddc483817582), Trace(trace_id=337f538f70a940b5ba8e6f93780a73ad), Trace(trace_id=cd2bb3cf73cd444699ac6f25176ad66f), Trace(trace_id=f982f5b92cc540a5bc47e77bc0553fc8), Trace(trace_id=f62cd068244c4789992a9a555e54988f), Trace(trace_id=75959e0765cf4147bbc013e3d33773be), Trace(trace_id=89d48641f2844f8c9d5accba99a6f2f0), Trace(trace_id=a787321988444bee9c76a4f0ae5cadb2)]